In [1]:
from collections import OrderedDict    # For recording the model specification 

import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations

import pylogit as pl                   # For choice model estimation
import pickle

In [2]:
catsup = pd.read_csv('data/catsup_trainformat.csv')
catsup.head()

,chosen,id,alt,display,feature,price,chid
0,0,1,0,0,0,4.6,0
1,0,1,1,0,0,3.7,0
2,1,1,2,0,0,5.2,0
3,0,1,3,0,0,3.4,0
4,0,1,0,0,0,4.6,1


In [10]:
infile = open('./data/10_dgp.p', 'rb')
big_dict = pickle.load(infile)
Y_dgp = big_dict['theta: [ 1.5  1.  -1.1  0.8  0.1  1.2]']
print(Y_dgp.shape)
chosen = np.zeros([11192,1])
j = 0
for choice in Y_dgp[:,0]:
    for i in range(4):
        if i == choice:
            chosen[i+j] = True
        else:
            chosen[i+j] = False
    j+=4
catsup['chosen'] = chosen
catsup.head(30)

(2798, 10)


,chosen,id,alt,display,feature,price,chid
0,0.0,1,0,0,0,4.6,0
1,0.0,1,1,0,0,3.7,0
2,0.0,1,2,0,0,5.2,0
3,1.0,1,3,0,0,3.4,0
4,0.0,1,0,0,0,4.6,1
5,1.0,1,1,0,0,4.3,1
6,0.0,1,2,0,0,5.2,1
7,0.0,1,3,0,0,4.4,1
8,0.0,1,0,0,0,4.6,2
9,1.0,1,1,0,1,2.5,2


In [3]:
#get variable list
index_var_names = ['display','feature','price']
for col in index_var_names:
    catsup[col] = catsup[col].astype(float)

In [4]:
#specification
example_specification = OrderedDict()
example_names = OrderedDict()

# Note that the names used below are simply for consistency with
# the coefficient names given in the mlogit vignette.
for col in index_var_names:
    example_specification[col] = [[0, 1, 2, 3]]
    example_names[col] = [col]

In [5]:
# Provide the module with the needed input arguments to create
# an instance of the Mixed Logit model class.

# Note that "chid" is used as the obs_id_col because "chid" is
# the choice situation id.

# Currently, the obs_id_col argument name is unfortunate because
# in the most general of senses, it refers to the situation id.
# In panel data settings, the mixing_id_col argument is what one 
# would generally think of as a "observation id".

# For mixed logit models, the "mixing_id_col" argument specifies
# the units of observation that the coefficients are randomly
# distributed over.
infile = open('./data/500_MC_dgp_uts.p', 'rb')
big_dict = pickle.load(infile)
Y_dgp = big_dict['theta: [ 1.5  1.  -1.1  0.4  0.1  0.6]']
print(Y_dgp.shape)
results = []
for row in range(100):
    chosen = np.zeros([11192,1])
    j = 0
    for choice in Y_dgp[:,row]:
        for i in range(4):
            if i == choice:
                chosen[i+j] = True
            else:
                chosen[i+j] = False
        j+=4
    catsup['chosen'] = chosen
    example_mixed = pl.create_choice_model(data=catsup,
                                       alt_id_col="alt",
                                       obs_id_col="chid",
                                       choice_col="chosen",
                                       specification=example_specification,
                                       model_type="Mixed Logit",
                                       names=example_names,
                                       mixing_id_col="id",
                                       mixing_vars=index_var_names)

    # Note 2 * len(index_var_names) is used because we are estimating
    # both the mean and standard deviation of each of the random coefficients
    # for the listed index variables.
    example_mixed.fit_mle(init_vals=np.zeros(2 * len(index_var_names)),
                          num_draws=250,
                          seed=10)

    # Look at the estimated results
    print(example_mixed.get_statsmodels_summary())
    results.append(example_mixed.get_statsmodels_summary())

(2798, 500)
Log-likelihood at zero: -3,878.8516
Initial Log-likelihood: -3,878.8516


C:\Users\niels\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:507: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 25.33 seconds.
Final log-likelihood: -2,427.0852


C:\Users\niels\Anaconda3\lib\site-packages\pylogit\base_multinomial_cm_v2.py:1259: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.robust_cov)),
C:\Users\niels\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\niels\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\niels\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


                     Mixed Logit Model Regression Results                     
Dep. Variable:                 chosen   No. Observations:                2,798
Model:              Mixed Logit Model   Df Residuals:                    2,792
Method:                           MLE   Df Model:                            6
Date:                Thu, 27 Jun 2019   Pseudo R-squ.:                   0.374
Time:                        13:24:10   Pseudo R-bar-squ.:               0.373
AIC:                        4,866.170   Log-Likelihood:             -2,427.085
BIC:                        4,901.790   LL-Null:                    -3,878.852
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
display           1.1759      0.111     10.600      0.000       0.958       1.393
feature           0.9422      0.132      7.151      0.000       0.684       1.200
price            -1.6252      0.052    -

...

Estimation Time for Point Estimation: 16.25 seconds.
Final log-likelihood: -3,034.7439
                     Mixed Logit Model Regression Results                     
Dep. Variable:                 chosen   No. Observations:                2,798
Model:              Mixed Logit Model   Df Residuals:                    2,792
Method:                           MLE   Df Model:                            6
Date:                Thu, 27 Jun 2019   Pseudo R-squ.:                   0.218
Time:                        14:04:58   Pseudo R-bar-squ.:               0.216
AIC:                        6,081.488   Log-Likelihood:             -3,034.744
BIC:                        6,117.108   LL-Null:                    -3,878.852
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
display           1.1673      0.093     12.548      0.000       0.985       1.350
feature           1.2300      0.108

C:\Users\niels\Anaconda3\lib\site-packages\pylogit\base_multinomial_cm_v2.py:1259: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.robust_cov)),
C:\Users\niels\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\niels\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\niels\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


                     Mixed Logit Model Regression Results                     
Dep. Variable:                 chosen   No. Observations:                2,798
Model:              Mixed Logit Model   Df Residuals:                    2,792
Method:                           MLE   Df Model:                            6
Date:                Thu, 27 Jun 2019   Pseudo R-squ.:                   0.283
Time:                        14:06:04   Pseudo R-bar-squ.:               0.282
AIC:                        5,573.104   Log-Likelihood:             -2,780.552
BIC:                        5,608.724   LL-Null:                    -3,878.852
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
display           2.2409      0.149     15.002      0.000       1.948       2.534
feature           1.0501      0.121      8.703      0.000       0.814       1.287
price            -0.9805      0.038    -

C:\Users\niels\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:507: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 19.64 seconds.
Final log-likelihood: -2,786.5975
                     Mixed Logit Model Regression Results                     
Dep. Variable:                 chosen   No. Observations:                2,798
Model:              Mixed Logit Model   Df Residuals:                    2,792
Method:                           MLE   Df Model:                            6
Date:                Thu, 27 Jun 2019   Pseudo R-squ.:                   0.282
Time:                        14:06:26   Pseudo R-bar-squ.:               0.280
AIC:                        5,585.195   Log-Likelihood:             -2,786.598
BIC:                        5,620.815   LL-Null:                    -3,878.852
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
display           1.1465      0.102     11.206      0.000       0.946       1.347
feature           1.1108      0.120

In [9]:
pickle.dump()